### Lab Complete Song Recommender

In [1]:
import config
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pickle
from IPython.display import IFrame

#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_se))

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!")
        
with open("scaler.pickle", "rb") as f:
    scaler=pickle.load(f)

with open("kmeans.pickle", "rb") as f:
    model=pickle.load(f)

In [4]:
music_df2= pd.read_csv('Data/clustered_music_features.csv')
#music_df2.dtypes
music_df2.cluster.unique()


#top100 = pd.read_csv('billboard100.csv')

array([3, 2, 7, 6, 5, 1, 0, 4], dtype=int64)

In [5]:
usersong_input = (input("Which song do you like?  ").replace(" ","").lower() or input('Which song do you like?  '))

usersong = sp.search(q=usersong_input, limit=1 ,market="GB")


#song_row = top100[top100["song"].str.contains(usersong_input)]
#if len(song_row) == 0:
    #print("The song you pick is not hot.")
#else:
check_song = input("Did you mean " + usersong["tracks"]["items"][0]['name'] + " by " + usersong["tracks"]['items'][0]['artists'][0]['name'] + '?  ')

if check_song in ('yes', 'y', 'Y', 'YES'):
    usersong_all=sp.audio_features(usersong["tracks"]["items"][0]["uri"])[0]
    usersong_all=pd.DataFrame([usersong_all])
    usersong_features=usersong_all.drop(columns=['id','uri','track_href', 'analysis_url', 'type', 'speechiness'])
    usersong_features_scaled=scaler.transform(usersong_features)

    cluster_no= int(model.predict(usersong_features_scaled))
    print('cluster number: ',  cluster_no)
    recommendation_df =  music_df2[music_df2['cluster']==cluster_no]
    recommended_song = recommendation_df.sample(1)
            
        
    print(('You might also like ')
            + ('\033[1m')
            + (recommended_song['title'].values[0])
            + ('\033[0m') + (' by ')
            + (recommended_song['all_artists'].values[0]))
    
    track_id_recommended = recommended_song['id'].values[0]
       
else:
    
     print("I don't know that song.")

Which song do you like?  paint it
Did you mean Paint It, Black by The Rolling Stones?  y
cluster number:  7
You might also like A Song For The Lovers by Richard Ashcroft


In [6]:
print('You can listen to this song recommendation here: \n')

from IPython.display import IFrame
track_id = track_id_recommended
IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

You can listen to this song recommendation here: 

